In [1]:
import pandas as pd
import numpy as np
import re


# Chuẩn bị dữu liệu
# Load các file dữ liệu đã làm sạch
df_ung_vien = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_candidate_profile.csv')
df_diem = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_interview_score.csv')
df_luong = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_salary_offer.csv')

In [4]:
# NHIỆM VỤ 1 – CLEAN DAT

for df in [df_ung_vien, df_diem, df_luong]:
    df["candidate_id"] = df["candidate_id"].astype(str).str.strip().str.upper()

# ---- full_name (nếu tồn tại) ----
if "full_name" in df_ung_vien.columns:
    df_ung_vien["full_name"] = (
        df_ung_vien["full_name"]
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
        .str.title()
    )

# ---- experience_years ----
def clean_exp(x):
    if pd.isna(x):
        return np.nan
    x = re.sub(r"[^\d\-]", "", str(x))
    try:
        x = int(x)
        return x if x >= 0 else np.nan
    except:
        return np.nan

if "experience_years" in df_ung_vien.columns:
    df_ung_vien["experience_years"] = df_ung_vien["experience_years"].apply(clean_exp)

# ---- round & score ----
df_diem["round"] = (
    df_diem["round"]
    .astype(str)        # Ép toàn bộ sang string
    .str.strip()
    .str.title()
)
df_diem["score"] = pd.to_numeric(df_diem["score"], errors="coerce")
df_diem.loc[(df_diem["score"] < 0) | (df_diem["score"] > 10), "score"] = np.nan

# ---- offer_salary ----
def clean_salary(x):
    if pd.isna(x):
        return np.nan
    x = str(x).lower().replace("tr", "000000")
    x = re.sub(r"[^\d]", "", x)
    return int(x) if x else np.nan

df_luong["offer_salary"] = df_luong["offer_salary"].apply(clean_salary)

# ---- position ----
def clean_position(x):
    x = str(x).lower()
    if "analyst" in x:
        return "Data Analyst"
    if "python" in x:
        return "Python Developer"
    return "Developer"

df_luong["position"] = df_luong["position"].apply(clean_position)

# ---- SAVE CLEAN FILES ----
df_ung_vien.to_csv("candidate_profile_clean.csv", index=False)
df_diem.to_csv("interview_score_clean.csv", index=False)
df_luong.to_csv("salary_offer_clean.csv", index=False)

In [5]:
# NHIỆM VỤ 2 – Truy vấn và thống kê mô tả

# Số ứng viên theo vị trí
candidate_by_position = df_luong["position"].value_counts()

# Ứng viên thiếu điểm
missing_score = df_diem[df_diem["score"].isna()]

# Điểm trung bình theo vòng
avg_score_by_round = df_diem.groupby("round")["score"].mean()

# Có kinh nghiệm nhưng điểm thấp
exp_low_score = df_diem[
    (df_diem["score"] < 5)
].merge(df_ung_vien, on="candidate_id")

# Số ứng viên có offer
offer_count = df_luong["offer_salary"].notna().sum()

print("Số ứng viên theo vị trí:")
print(candidate_by_position.reset_index(name="candidate"))
print("Điểm trung bình theo vòng:")
print(avg_score_by_round.reset_index(name="score").to_string(index=False))
if exp_low_score.empty:
    print("Không có ứng viên có kinh nghiệm nhưng điểm thấp")
else:
    print("Ứng viên có kinh nghiệm nhưng điểm thấp:")
    print(exp_low_score)
print("Số ứng viên có offer:")
print(offer_count)
print("Ứng viên thiếu điểm:")
print(missing_score)

Số ứng viên theo vị trí:
           position  candidate
0      Data Analyst         14
1  Python Developer          9
2         Developer          2
Điểm trung bình theo vòng:
round    score
    1 7.266667
    2 6.333333
    3 7.071429
Không có ứng viên có kinh nghiệm nhưng điểm thấp
Số ứng viên có offer:
17
Ứng viên thiếu điểm:
Empty DataFrame
Columns: [candidate_id, round, score]
Index: []
